In [2]:
import pandas as pd
ER2017 = pd.read_csv("UG2017.csv", sep = ";") #Election Results 2017
ER2012 = pd.read_csv("UG2012.csv", sep = ";") #Election Results 2012
ER2010 = pd.read_csv("UG2010.csv", sep = ";") #Election Results 2010
ER2006 = pd.read_csv("UG2006.csv", sep = ";") #Election Results 2006
ER2003 = pd.read_csv("UG2003.csv", sep = ";") #Election Results 2003
ER2002 = pd.read_csv("UG2002.csv", sep = ";") #Election Results 2002
ER1998 = pd.read_csv("UG1998.csv", sep = ";") #Election Results 1998

## Deleting Caribbean Netherlands, added in 2012 as municipalities (in 2017 as Bonaire)
for i,j in enumerate(ER2012["OuderRegioNaam"]):
    if j == 'Niet-provinciaal ingedeeld':
        ER2012 = ER2012.drop([i])
for i,j in enumerate(ER2017["OuderRegioNaam"]):
    if j == 'Bonaire':
        ER2017 = ER2017.drop([i])






In [3]:
munchange = []
for year in range(1999,2020):
    if year < 2018:
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ";")  # For every csv file
    else: 
        municipalchanges = pd.read_csv("GI_{}.csv".format(year), sep = ",")  # For every csv file
    ## Drop row provence change (provence change is not necessary for dataframe)
    if year == 2002:
        municipalchanges = municipalchanges.drop([13])
    #Change codes in csv file to match electionresult files
    for i,j in enumerate(municipalchanges["Gemeente Code Old"]):
        if j <= 10:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G000{}'.format(j)
        elif j <= 100:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code Old"] = 'G{}'.format(j)
    for i,j in enumerate(municipalchanges["Gemeente Code New"]):
        if j <= 100:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G00{}'.format(j)
        elif j <= 1000:
            municipalchanges.loc[i,"Gemeente Code New"] = 'G0{}'.format(j)
        else: 
            municipalchanges.loc[i,"Gemeente Code New"] = 'G{}'.format(j)
    munchange.append(municipalchanges)
    
    

In [5]:
electionyears = [[ER1998,['Partij van de Arbeid (P.v.d.A.)'] ,"1998_1"],
                 [ER2002,['Partij van de Arbeid (P.v.d.A.)'] ,"1998_2"],
                 [ER2002,['CDA'],"2002_1"],
                 [ER2003,['CDA'],"2002_2"],
                 [ER2003,['CDA'],"2003_1"],
                 [ER2006,['CDA'],"2003_2"],
                 [ER2006,['CDA'],"2006_1"],       
                 [ER2010,['CDA'],"2006_2"],
                 [ER2010,['VVD'],"2010_1"],
                 [ER2012,['VVD'],"2010_2"],
                 [ER2012,['VVD'],"2012_1"],
                 [ER2017,['VVD'],"2012_2"]]

In [6]:
### 
electionresults = []
electionresultpercent = []

for election in electionyears:
    electionfileoriginal = election[0]
    kabinet = election[1]
    electionfile = electionfileoriginal[["RegioNaam","RegioCode","GeldigeStemmen"]]
    electionfile["Totvotincpar"] =  electionfileoriginal[kabinet].sum(axis = 1)
    

    electionyear = election[2]
    for Municipalchanges in munchange:
    #     display(Municipalchanges)
    #     Computing old values to new
        Regioncodeoldlist = Municipalchanges['Gemeente Code Old'].tolist()
        for i,j in enumerate(electionfile["RegioCode"]):
            if j in Regioncodeoldlist:
                for index, oldcode in enumerate(Municipalchanges["Gemeente Code Old"]):
                    if oldcode == j:
                        electionfile.iloc[i,1] = Municipalchanges.iloc[index, 2]
                        electionfile.iloc[i,0] = Municipalchanges.iloc[index, 3]


    electionfile["geldigestemmen{}".format(electionyear)] = electionfile.groupby(['RegioNaam'])['GeldigeStemmen'].transform('sum')
    electionfile["totalvotesincumbent{}".format(electionyear)] = electionfile.groupby(['RegioNaam'])['Totvotincpar'].transform('sum')
    electionfile = electionfile.drop_duplicates(subset=['RegioCode'])
    electionfile = electionfile.drop(['GeldigeStemmen','Totvotincpar'],axis = 1)
    electionfile["Percent incumbent{}".format(electionyear)] = electionfile["totalvotesincumbent{}".format(electionyear)]/electionfile['geldigestemmen{}'.format(electionyear)]
    electionresults.append(electionfile)
    electionresultpercent.append(electionfile[["Percent incumbent{}".format(electionyear),"RegioCode"]])


for i,k in enumerate(electionresults):
    if i ==0: 
        result = electionresults[0]
    else:
        result = result.merge(electionresults[i], how="right", on=["RegioNaam","RegioCode"])

display(result)




<ipython-input-6-c44ea31b19d8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electionfile["Totvotincpar"] =  electionfileoriginal[kabinet].sum(axis = 1)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
<ipython-input-6-c44ea31b19d8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,RegioNaam,RegioCode,geldigestemmen1998_1,totalvotesincumbent1998_1,Percent incumbent1998_1,geldigestemmen1998_2,totalvotesincumbent1998_2,Percent incumbent1998_2,geldigestemmen2002_1,totalvotesincumbent2002_1,...,Percent incumbent2010_1,geldigestemmen2010_2,totalvotesincumbent2010_2,Percent incumbent2010_2,geldigestemmen2012_1,totalvotesincumbent2012_1,Percent incumbent2012_1,geldigestemmen2012_2,totalvotesincumbent2012_2,Percent incumbent2012_2
0,Aa en Hunze,G1680,15588,6424,0.412112,16206,4411,0.272183,16206,2606,...,0.253991,16342,4725,0.289132,16342,4725,0.289132,17215,4268,0.247923
1,Altena,G1959,29698,6396,0.215368,32759,3051,0.093135,32759,12056,...,0.166033,31745,8171,0.257395,31745,8171,0.257395,35036,6600,0.188378
2,Aalsmeer,G0358,13529,2866,0.211841,14704,1486,0.101061,14704,4590,...,0.345294,17636,8058,0.456906,17636,8058,0.456906,19443,6764,0.347889
3,Aalten,G0197,15495,4065,0.262343,16423,2124,0.129331,16423,8247,...,0.153903,15880,3485,0.219458,15880,3485,0.219458,17317,3324,0.191950
4,De Ronde Venen,G0736,24975,4596,0.184024,27107,2367,0.087321,27107,8696,...,0.331752,26038,11677,0.448460,26038,11677,0.448460,28467,9331,0.327783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Zoetermeer,G0637,59407,15760,0.265289,65922,9142,0.138679,65922,15158,...,0.247988,68769,20448,0.297343,68769,20448,0.297343,75877,17595,0.231888
352,Zoeterwoude,G0638,5001,1153,0.230554,5315,493,0.092756,5315,2079,...,0.237788,5026,1662,0.330680,5026,1662,0.330680,5581,1450,0.259810
353,Tynaarlo,G1730,19274,6186,0.320951,20272,4223,0.208317,20272,4037,...,0.236470,20431,5727,0.280309,20431,5727,0.280309,22356,5283,0.236312
354,Zundert,G0879,10032,2771,0.276216,11180,1186,0.106082,11180,4370,...,0.237754,11287,4034,0.357402,11287,4034,0.357402,13112,3746,0.285692


In [7]:
dfpercentincumbent = pd.DataFrame()

for i,k in enumerate(electionresultpercent):
    if i ==0: 
        dfpercentincumbent = electionresultpercent[0]
    else:
        dfpercentincumbent = dfpercentincumbent.merge(electionresultpercent[i], how="outer", on=["RegioCode"])

display(dfpercentincumbent)

,Percent incumbent1998_1,RegioCode,Percent incumbent1998_2,Percent incumbent2002_1,Percent incumbent2002_2,Percent incumbent2003_1,Percent incumbent2003_2,Percent incumbent2006_1,Percent incumbent2006_2,Percent incumbent2010_1,Percent incumbent2010_2,Percent incumbent2012_1,Percent incumbent2012_2
0,0.412112,G1680,0.272183,0.160805,0.148417,0.148417,0.152272,0.152272,0.078821,0.253991,0.289132,0.289132,0.247923
1,0.215368,G1959,0.093135,0.368021,0.414248,0.414248,0.380986,0.380986,0.227825,0.166033,0.257395,0.257395,0.188378
2,0.211841,G0358,0.101061,0.312160,0.350549,0.350549,0.347924,0.347924,0.156941,0.345294,0.456906,0.456906,0.347889
3,0.262343,G0197,0.129331,0.502162,0.489825,0.489825,0.410860,0.410860,0.268865,0.153903,0.219458,0.219458,0.191950
4,0.184024,G0736,0.087321,0.320803,0.355287,0.355287,0.364714,0.364714,0.162498,0.331752,0.448460,0.448460,0.327783
...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,0.265289,G0637,0.138679,0.229938,0.223943,0.223943,0.227165,0.227165,0.088228,0.247988,0.297343,0.297343,0.231888
352,0.230554,G0638,0.092756,0.391157,0.391882,0.391882,0.382191,0.382191,0.198426,0.237788,0.330680,0.330680,0.259810
353,0.320951,G1730,0.208317,0.199142,0.192534,0.192534,0.176024,0.176024,0.099917,0.236470,0.280309,0.280309,0.236312
354,0.276216,G0879,0.106082,0.390877,0.423621,0.423621,0.398094,0.398094,0.230948,0.237754,0.357402,0.357402,0.285692


In [8]:
# for index, column in enumerate(dfpercentincumbent):
#     if column == "RegioCode":
#         continue
#     for row in dfpercentincumbent.index.values:



dfpercentincumbent["Election02"]= (dfpercentincumbent["Percent incumbent1998_2"]-dfpercentincumbent["Percent incumbent1998_1"])/dfpercentincumbent["Percent incumbent1998_1"]
dfpercentincumbent["Election03"]= (dfpercentincumbent["Percent incumbent2002_2"]-dfpercentincumbent["Percent incumbent2002_1"])/dfpercentincumbent["Percent incumbent2002_1"]
dfpercentincumbent["Election06"]= (dfpercentincumbent["Percent incumbent2003_2"]-dfpercentincumbent["Percent incumbent2003_1"])/dfpercentincumbent["Percent incumbent2003_1"]
dfpercentincumbent["Election10"]= (dfpercentincumbent["Percent incumbent2006_2"]-dfpercentincumbent["Percent incumbent2006_1"])/dfpercentincumbent["Percent incumbent2006_1"]
dfpercentincumbent["Election12"]= (dfpercentincumbent["Percent incumbent2010_2"]-dfpercentincumbent["Percent incumbent2010_1"])/dfpercentincumbent["Percent incumbent2010_1"]
dfpercentincumbent["Election17"]= (dfpercentincumbent["Percent incumbent2012_2"]-dfpercentincumbent["Percent incumbent2012_1"])/dfpercentincumbent["Percent incumbent2012_1"]
dfpercentincumbent = dfpercentincumbent.drop(["Percent incumbent1998_1","Percent incumbent1998_2","Percent incumbent2002_1","Percent incumbent2002_2","Percent incumbent2003_1","Percent incumbent2003_2","Percent incumbent2006_1","Percent incumbent2006_2","Percent incumbent2010_1","Percent incumbent2010_2","Percent incumbent2012_1","Percent incumbent2012_2"],axis = 1)

In [9]:
dfpercentincumbent

,RegioCode,Election02,Election03,Election06,Election10,Election12,Election17
0,G1680,-0.339541,-0.077032,0.025973,-0.482371,0.138356,-0.142526
1,G1959,-0.567556,0.125609,-0.080293,-0.402013,0.550260,-0.268137
2,G0358,-0.522940,0.122979,-0.007488,-0.548921,0.323238,-0.238600
3,G0197,-0.507016,-0.024568,-0.161210,-0.345604,0.425948,-0.125346
4,G0736,-0.525493,0.107493,0.026533,-0.554451,0.351794,-0.269092
...,...,...,...,...,...,...,...
351,G0637,-0.477252,-0.026073,0.014387,-0.611613,0.199025,-0.220132
352,G0638,-0.597680,0.001853,-0.024729,-0.480821,0.390655,-0.214317
353,G1730,-0.350938,-0.033180,-0.085752,-0.432368,0.185393,-0.156959
354,G0879,-0.615945,0.083772,-0.060259,-0.419865,0.503244,-0.200642


In [44]:
# result.to_csv('/Users/ruben/Documents/Datascience & Society/Thesis/Electionresults98-17.csv')
dfpercentincumbent.to_csv('/Users/ruben/Documents/Datascience & Society/Thesis/VERSION_Election_WinPart.csv')